# The DSS Jupyter Form

Please execute the box below to create the universal download form. You can then make a selection and then request the data by exectuing the code cell below that.

<div class="alert alert-block alert-warning">
<b>Prototype:</b> This is using a very new package which is not yet fully supported by ECMWF.
</div>


In [1]:
!pip install -q ecmwf-jupyter-components
from ecmwf.jupyter_components.jupyter_forms import DssDownloadForm

download_form = DssDownloadForm()

[2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
[2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Output()

In [2]:
download_form.collection_id, download_form.request

('reanalysis-era5-land',
 {'variable': ['2m_temperature'],
  'year': ['2024'],
  'month': ['01'],
  'day': ['01'],
  'time': ['00:00', '06:00', '12:00', '18:00']})

## Use earthkit to access the data

We can use this request selection to access the data with Earthkit and load into our interactive session. Below we make the data request and "describe" the object that we have downloaded.

In [10]:
from earthkit import data as ek_d

eds = ek_d.from_source("cds", download_form.collection_id, download_form.request)

eds.describe()

2025-05-16 16:41:30,915 WARNING [2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
2025-05-16 16:41:30,916 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-16 16:41:30,916 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


,,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,
2t,surface,0,"20240101,20231231",0,"24,18,...",167,l5,oper,fc,0001


We can use earthkit to convert the data to an xarray object we try with default options, and if that fails we try with an alternative `time_dim_mode='valid_time'`. Modifying the `time_dim_mode` is useful for opening grib data where the time dimensions may not produce a complete hypercube. Note that this may not work every dataset available accross the Data Store Service (DSS).

In [11]:
# 
try:
    xr_ds = eds.to_xarray()
except ValueError:
    xr_ds = eds.to_xarray(time_dim_mode="valid_time")
xr_ds

<xarray.Dataset> Size: 208MB
Dimensions:     (valid_time: 4, latitude: 1801, longitude: 3600)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 32B 2024-01-01 ... 2024-01-01T18:...
  * latitude    (latitude) float64 14kB 90.0 89.9 89.8 ... -89.8 -89.9 -90.0
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
Data variables:
    2t          (valid_time, latitude, longitude) float64 207MB ...
Attributes: (12/13)
    param:        2t
    paramId:      167
    class:        l5
    stream:       oper
    levtype:      sfc
    type:         fc
    ...           ...
    date:         20231231
    time:         0
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF